In [ ]:
#!/usr/bin/env python

import pandas as pd
import networkx as nx
from networkx.algorithms.bipartite.matrix import biadjacency_matrix
import numpy as np
from sklearn.metrics import precision_recall_curve, auc
import random
from sklearn import metrics
import time
import numpy.linalg as LA
from scipy.linalg import inv
from joblib import Parallel, delayed
from math import sqrt
from sklearn.utils import parallel_backend
from itertools import product
from sklearn.metrics import roc_auc_score
from scipy.spatial.distance import cdist

random.seed(1949) # for dataset split
np.random.seed(1949) # for matrix initialization

This file is for the study of ADRs prediction using VKR. After the function in section 1 run, nested CV, CV and result of test set in section 2 can be ran separately.

## 1. Define the functions used for VKR

In [ ]:
def option(str):
    global methodOption
    methodOption = str

### Functions for VKR

In [ ]:
def NMF(bipart_graph, component, WMK, lmd, max_iter, tolerance=1/1000000):
    np.random.seed(1949)
    random.seed(1949)

    W = WMK.copy()
    X = bipart_graph.copy()
    m, n = X.shape
    k = component
 
    D = np.matrix(np.diag(np.asarray(W.copy()).sum(axis=1)))
    L = D.copy() - W.copy()

    # Initialize U & V

    U = np.random.random((m, k))
    V = np.random.random((n, k))

    # Updating U V
    eps = 2**-8

    term1 = LA.norm(X - np.dot(U, V.T))**2
    term2 = lmd * np.trace(np.dot(np.dot(V.T, L), V))
    Obj0 = term1 + term2
    Obj1 = Obj0


    for i in range(max_iter):
        XV = np.dot(X, V)
        UVtV = np.dot(np.dot(U, V.T), V) + eps

        U *= XV
        U /= UVtV
        
        XtU_lmdWV = np.dot(X.T, U) + lmd*np.dot(W, V)
        VUtU_lmdDV = np.dot(np.dot(V, U.T), U) + lmd*np.dot(D, V) + eps
        V *= XtU_lmdWV
        V /= VUtU_lmdDV

        # Objective function
        
        term1 = LA.norm(X - np.dot(U, V.T))**2
        term2 = lmd * np.trace(np.dot(np.dot(V.T, L), V))
        Obj2 = term1 + term2    
        ObjDiff = Obj1 - Obj2
        Obj1 = Obj2

        if(ObjDiff < (Obj0 *tolerance)):
            print("Converged in iteration: ", i, "ObjDiff: ", ObjDiff, "Obj: ", Obj2)
            return(U, V, np.dot(U, V.T))
        elif i == max_iter - 1:
            print("Has not converged, reach the maximum iteration")
            return(U, V, np.dot(U, V.T))

In [ ]:
def KernelRegression(matrix,feature_matrix1,feature_matrix2,idx_train,idx_test,l1,l2,s):
    sigma = s
    lmd1 = l1
    lmd2 = l2
        
    X1 = np.array(feature_matrix1[idx_train, :].copy()).tolist()
    X_new1 = np.array(feature_matrix1[idx_test, :].copy()).tolist()
    X2 = np.array(feature_matrix2[idx_train, :].copy()).tolist()
    X_new2 = np.array(feature_matrix2[idx_test, :].copy()).tolist()
    y = matrix[:, idx_train].copy()
    Y = pd.DataFrame(y.T.copy())
    matrix_new = matrix.copy().astype(float)

    distance1 = cdist(X_new1, X1)**2
    distance2 = cdist(X_new2, X2)**2

    kernel1 = np.exp(-distance1/sigma**2)
    kernel2 = np.exp(-distance2/sigma**2)

    similarity1 = cdist(X1, X1)**2
    similarity2 = cdist(X2, X2)**2

    K1 = pd.DataFrame(np.exp(-similarity1/sigma**2))
    K2 = pd.DataFrame(np.exp(-similarity2/sigma**2))

    n = len(idx_train) # size of known drug

    
    if methodOption == "KRNMF1":
        Lmd = np.diag(np.ones(n)*lmd1)
        W = inv(K1.dot(K1)+Lmd).dot(K1.dot(Y))
        y_new = kernel1.dot(W)
    elif methodOption == "KRNMF2":
        Lmd = np.diag(np.ones(n)*lmd1)
        W = inv(K2.dot(K2)+Lmd).dot(K2.dot(Y))
        y_new = kernel2.dot(W)
    elif methodOption == "KRNMF1&2":
        c1 = 0.5
        c2 = 0.5
        Lmd = np.diag(np.ones(n)*lmd1)
        K = c1*K1 + c2*K2
        W = inv(K.dot(K)+Lmd).dot(K.dot(Y))
        y_new = (c1*kernel1+c2*kernel2).dot(W)

    matrix_new[:, idx_test] = y_new.T
    return matrix_new

In [ ]:
def Adaptive(matrix,feature_matrix1,feature_matrix2,idx_train,idx_test,l1,l2,s,k):
    sigma = s
    lmd1 = l1
    lmd2 = l2

    X1 = np.array(feature_matrix1[idx_train, :].copy()).tolist()
    X_new1 = np.array(feature_matrix1[idx_test, :].copy()).tolist()
    X2 = np.array(feature_matrix2[idx_train, :].copy()).tolist()
    X_new2 = np.array(feature_matrix2[idx_test, :].copy()).tolist()
    y = matrix[:, idx_train].copy()
    matrix_new = matrix.copy().astype(float)

    similarity1 = cdist(X1, X1)**2
    WMK1 = pd.DataFrame(np.exp(-similarity1/sigma**2))
    similarity2 = cdist(X2, X2)**2
    WMK2 = pd.DataFrame(np.exp(-similarity2/sigma**2))


    m, n = matrix.shape
    Vout = np.zeros((n, k))

    if methodOption == "KRNMF1":
        U,V,preds = NMF(y, component=k, WMK=WMK1, lmd=lmd2, max_iter=10000)
        Vout[idx_train, :] = V
    elif methodOption == "KRNMF2":
        U,V,preds = NMF(y, component=k, WMK=WMK2, lmd=lmd2, max_iter=10000)
        Vout[idx_train, :] = V
    elif methodOption == "KRNMF1&2":
        c1=0.5
        c2=0.5
        U,V,preds = NMF(y, component=k, WMK=c1*WMK1+c2*WMK2, lmd=lmd2, max_iter=10000)
        Vout[idx_train, :] = V

    Vpreds = KernelRegression(Vout.T,feature_matrix1,feature_matrix2,idx_train,idx_test,l1,l2,s)


    preds = U.dot(Vpreds)
    
    return preds

### Function for generating features for common drugs

In [ ]:
def FeaturePreprocess(df_all, drug_nodes):
    
    drug_nodes_df = np.intersect1d(df_all.index, drug_nodes)
    df = df_all.loc[drug_nodes_df]
    _, q = df.shape
    drug_nodes_diff = np.setdiff1d(drug_nodes, (df.index).tolist())
    n = len(drug_nodes_diff)
    df_diff = pd.DataFrame(np.zeros(n*q).reshape(n,q))
    df_diff.index = drug_nodes_diff
    df_diff.columns = df.columns
    df_all = pd.concat([df, df_diff], axis = 0)
    featureMat = df_all.loc[drug_nodes]
    return np.array(featureMat)

### Function for cross validation and nested cross validation

In [ ]:
def fold(IDX1,IDX2,feature_matrix1,feature_matrix2,matrix,l1,l2,s,k):
    # IDX1 target index, need to be evaluated
    # IDX2 test index, masked

    target_idx = IDX1
    mask_idx = IDX2
    Ground_Truth = matrix.copy()
    side_effects_drug_relation_copy = matrix.copy()

    # target_idx = IDX2
    ### making all the links to predict as 0 ###############    
    for i in range(len(mask_idx)):
        side_effects_drug_relation_copy[:, mask_idx[i]] = 0
    
    m,n = side_effects_drug_relation_copy.shape

    drug_idx = list(range(n))
    existing_drug_idx = np.setdiff1d(drug_idx, mask_idx)
    
    print(methodOption + ' starts:')
    # real_stdout = sys.stdout
    # sys.stdout = open(os.devnull, "w")
    side_effects_drug_relation_fact = Adaptive(matrix=side_effects_drug_relation_copy,\
        feature_matrix1=feature_matrix1,feature_matrix2=feature_matrix2,idx_train=existing_drug_idx,idx_test=target_idx,l1=l1,l2=l2,s=s,k=k)
    # sys.stdout = real_stdout
    print(methodOption + ' ends:')


    # Set the out put of GNMF as prediction score
    score = side_effects_drug_relation_fact.copy()

    pr_auc_all = 0
    roc_auc_all = 0

    print("proportion of ground truth:", sum(Ground_Truth[:, target_idx].ravel())/(Ground_Truth[:, target_idx].shape[0]*Ground_Truth[:, target_idx].shape[1]))

    print('---evaluation---')


    prec, recall, threshold = precision_recall_curve(Ground_Truth[:, target_idx].ravel(), score[:, target_idx].ravel())
    pr_auc_all = auc(recall, prec)
    roc_auc_all = roc_auc_score(Ground_Truth[:, target_idx].ravel(), score[:, target_idx].ravel())



    print("-----")

    print("AUC-PR all:", pr_auc_all)

    print("-----")

    print("AUC-ROC all:", roc_auc_all)

    print("-----")


    return pr_auc_all, roc_auc_all

In [ ]:
def innerfold(IDX1,IDX2,feature_matrix1,feature_matrix2,matrix,l1,l2,s,k):
    # IDX1 target index, need to be evaluated
    # IDX2 test index, masked

    target_idx = IDX1
    mask_idx = IDX2
    Ground_Truth = matrix.copy()
    side_effects_drug_relation_copy = matrix.copy()
    ### making all the links to predict as 0 ###############    
    for i in range(len(mask_idx)):
        side_effects_drug_relation_copy[:, mask_idx[i]] = 0
    
    m,n = side_effects_drug_relation_copy.shape

    drug_idx = list(range(n))
    existing_drug_idx = np.setdiff1d(drug_idx, mask_idx)
    
    side_effects_drug_relation_fact = Adaptive(matrix=side_effects_drug_relation_copy,\
        feature_matrix1=feature_matrix1,feature_matrix2=feature_matrix2,idx_train=existing_drug_idx, idx_test=target_idx,l1=l1,l2=l2,s=s,k=k)

    score = side_effects_drug_relation_fact.copy()

    pr_auc_all = 0
    roc_auc_all = 0
    
    prec, recall, threshold = precision_recall_curve(Ground_Truth[:, target_idx].ravel(), score[:, target_idx].ravel())
    pr_auc_all = auc(recall, prec)

    return pr_auc_all, roc_auc_all

In [ ]:
def plotfold(IDX1,IDX2,feature_matrix1,feature_matrix2,matrix,l1,l2,s,k):
    # IDX1 target index, need to be evaluated
    # IDX2 test index, masked

    print('First few target index:', IDX1[0:10])
    print('First few mask index:', IDX2[0:10])

    target_idx = IDX1
    mask_idx = IDX2
    Ground_Truth = matrix.copy()
    side_effects_drug_relation_copy = matrix.copy()
    ### making all the links to predict as 0 ###############    
    for i in range(len(mask_idx)):
        side_effects_drug_relation_copy[:, mask_idx[i]] = 0
    
    m,n = side_effects_drug_relation_copy.shape

    drug_idx = list(range(n))
    existing_drug_idx = np.setdiff1d(drug_idx, mask_idx)
    
    print(methodOption + ' starts:')
    side_effects_drug_relation_fact = Adaptive(matrix=side_effects_drug_relation_copy,\
        feature_matrix1=feature_matrix1,feature_matrix2=feature_matrix2,idx_train=existing_drug_idx,idx_test=target_idx,l1=l1,l2=l2,s=s,k=k)
    print(methodOption + ' ends:')


    score = side_effects_drug_relation_fact.copy()


    print("proportion of ground truth:", sum(Ground_Truth[:, target_idx].ravel())/(Ground_Truth[:, target_idx].shape[0]*Ground_Truth[:, target_idx].shape[1]))

    print('---evaluation---')


    prec, recall, prthreshold = precision_recall_curve(Ground_Truth[:, target_idx].ravel(), score[:, target_idx].ravel())
    pr_auc_all = auc(recall, prec)
    fpr, tpr, rocthreshold = metrics.roc_curve(Ground_Truth[:, target_idx].ravel(), score[:, target_idx].ravel())
    roc_auc_all = auc(fpr, tpr)


    print("-----")

    print("AUC-PR all:", pr_auc_all)

    print("-----")

    print("AUC-ROC all:", roc_auc_all)

    print("-----")

    Out1 = pd.DataFrame([prec, recall, prthreshold])
    Out2 = pd.DataFrame([fpr, tpr, rocthreshold])
    return Out1, Out2

### Functions for assigning arguments of CV and nested CV, as well as finding the best hyperparameters

In [ ]:
def setvar_tune(size):
    # set var for hyper pars tuning size is the hyper par size ALL_...

    global ALL_AUCPR_all
    global ALL_AUROC_all

    ALL_AUCPR_all = np.zeros(size)
    ALL_AUROC_all = np.zeros(size)

In [ ]:
def setvar_cv(FOLDS):
    # set var for cv 

    global AUC_roc_all
    global AUC_pr_all
    
    AUC_roc_all = np.zeros(FOLDS)
    AUC_pr_all = np.zeros(FOLDS)

In [ ]:
def asgvar_tune(idx, results):
    # assign var for cv from results
    # f: size of hyper pars

    ALL_AUCPR_all[idx] = results[0]
    ALL_AUROC_all[idx] = results[1]


In [ ]:
def asgvar_cv(f, results):
    # assign var for cv from results
    # f: size of hyper pars

    AUC_pr_all[f] = results[0]
    AUC_roc_all[f] = results[1]

In [ ]:
def tuning_results(tuneVar):
    idx = np.argmax(ALL_AUCPR_all)
    Var = tuneVar[idx]
    Value = ALL_AUCPR_all[idx]

    print("best hyperpar: ", Var)
    print("AUPRC: ", Value)

    
    return Var, Value

In [ ]:
def setvar_besttune(innerfolds):
    global besttunevalue
    global besttunevar
    besttunevalue = np.zeros(innerfolds) # best metric value
    besttunevar = np.zeros(innerfolds) # the value of best var
    besttunevar = besttunevar.tolist()

In [ ]:
def asg_besttune(f, value, var):
    besttunevalue[f] = value
    besttunevar[f] = var

In [ ]:
def besttune():
    idx = np.argmax(besttunevalue)
    value = besttunevalue[idx]
    var = besttunevar[idx]
    return value, var

In [ ]:
def cv_results():
    
    print("Mean AUC_pr_all", AUC_pr_all.mean()," ", "Standard Deviation:", AUC_pr_all.std())
    print("Mean AUC_roc_all", AUC_roc_all.mean()," ", "Standard Deviation:", AUC_roc_all.std())
    print("-----------")
    results = np.array([AUC_pr_all, AUC_roc_all])

    return results

### Function for parallel computation

In [ ]:
def tuning_loop(innermatrix, idx_train_inner, idx_test_inner, feature_matrix_inner1, feature_matrix_inner2, hyperparList, i):
    
    l1,l2,s,k = hyperparList[i]
    idx_target_inner = idx_test_inner
    # print('target size:', len(idx_target_inner))
    results = innerfold(idx_target_inner,idx_test_inner,feature_matrix1=feature_matrix_inner1,feature_matrix2=feature_matrix_inner2,matrix=innermatrix,l1=l1,l2=l2,s=s,k=k)
    asgvar_tune(i, results=results)
    # print("------ lmdKR: ", l1, "lmdNMF: ", l2, "sigma: ", s, "k: ", k, "------")

In [ ]:
def main(method_option,normalization=True,Validation=False,sets="intersect", l1=0.5, l2=0, s=0.5, k=20):
    random.seed(1949) # for dataset split
    np.random.seed(1949) # for matrix initialization
    option(method_option)

    random.seed(1949) # for dataset split
    np.random.seed(1949) # for matrix initialization
    df = pd.read_csv("data/side-effect-and-drug_name_upper.tsv",sep = "\t")
    drug_id = df["drugbank_id"] # put col of df in var
    drug_name = df["drugbank_name"]
    side_effect = df["side_effect_name"]
    
    
    edgelist1 = zip(side_effect, drug_name)
    ##making Biparite Graph##
    B = nx.DiGraph()
    B.add_nodes_from(side_effect,bipartite = 0)
    B.add_nodes_from(drug_name,bipartite = 1)
    B.add_edges_from(edgelist1)
    # B.add_weighted_edges_from(edgelist2)
    drug_nodes = {n for n, d in B.nodes(data=True) if d['bipartite']==1}
    side_effect_nodes = {n for n, d in B.nodes(data=True) if d['bipartite']==0}
    drug_nodes = list(drug_nodes)
    drug_nodes.sort()
    side_effect_nodes = list(side_effect_nodes)
    side_effect_nodes.sort()
    ###Getting the Bi-Adjacency matrix between side effects and drugs ###################
    matrix_all = biadjacency_matrix(B, row_order = side_effect_nodes, column_order = drug_nodes) # create biadjacency matrix for drug side effect graph
    matrix_all = matrix_all.A
    m_all,n_all = matrix_all.shape # number of side effect # number of drug
    
    
    ### Setting validation set / training set / testing set ###
    validate_sz = int(0.25 * n_all)
    IDX_all = list(range(n_all))
    random.shuffle(IDX_all)
    IDX_validate = sorted(IDX_all[0:validate_sz])
    print("first few validation set idx:")
    print(IDX_validate[0:10])
    IDX_validate_diff = np.setdiff1d(IDX_all, IDX_validate)
    matrix = matrix_all[:, IDX_validate_diff].copy()
    # featureMat1 = featureMat1_all[IDX_validate_diff, :][:, IDX_validate_diff].copy()
    # featureMat2 = featureMat2_all[IDX_validate_diff, :][:, IDX_validate_diff].copy()
    # print("WMK shape:")
    # print(featureMat1.shape)
    
    df1 = pd.read_csv("data/intersection_DGIdb_mat.tsv",sep = "\t")
    df2 = pd.read_csv("data/intersection_Fingerprint_mat.tsv",sep = "\t")
    featureMat1_all = FeaturePreprocess(df1, drug_nodes=drug_nodes)
    featureMat2_all = FeaturePreprocess(df2, drug_nodes=drug_nodes)
    # drug_nodes_feature1 = featureMat1_all.index
    # drug_nodes_feature2 = featureMat2_all.index
    featureMat1 = featureMat1_all[IDX_validate_diff, :].copy()
    featureMat2 = featureMat2_all[IDX_validate_diff, :].copy()
    
    
    non_zero_idx_union = np.hstack(np.where(~((featureMat1.sum(1) == 0) & (featureMat2.sum(1) == 0))))
    non_zero_idx_missing = np.hstack(np.where(~(~(featureMat1.sum(1) == 0) & ~(featureMat2.sum(1) == 0))))
    non_zero_idx_intersect = np.hstack(np.where(~(featureMat1.sum(1) == 0) & ~(featureMat2.sum(1) == 0)))
    if sets == "union":
        # union
        matrix = matrix[:, non_zero_idx_union].copy()
        featureMat1 = featureMat1[non_zero_idx_union, :].copy()
        featureMat2 = featureMat2[non_zero_idx_union, :].copy()
    elif sets == "intersect":
        # intersect
        non_zero_idx_intersect_all = np.hstack(np.where(~(featureMat1_all.sum(1) == 0) & ~(featureMat2_all.sum(1) == 0)))
    
        matrix_all = matrix_all[:, non_zero_idx_intersect_all].copy()
        featureMat1_all = featureMat1_all[non_zero_idx_intersect_all, :].copy()
        featureMat2_all = featureMat2_all[non_zero_idx_intersect_all, :].copy()
    
        matrix = matrix[:, non_zero_idx_intersect].copy()
        featureMat1 = featureMat1[non_zero_idx_intersect, :].copy()
        featureMat2 = featureMat2[non_zero_idx_intersect, :].copy()
    
        IDX_validate = np.setdiff1d(non_zero_idx_intersect_all, IDX_validate_diff)
        IDX_validate_diff = np.setdiff1d(non_zero_idx_intersect_all, IDX_validate)
    
        drug_nodes_intersect_all = np.array(drug_nodes)[non_zero_idx_intersect_all]
        drug_nodes_intersect_validate_diff = np.array(drug_nodes)[IDX_validate_diff]
        drug_nodes_intersect_validate = np.array(drug_nodes)[IDX_validate]
    
        IDX_validate = np.array([x for x in range(len(drug_nodes_intersect_all)) if drug_nodes_intersect_all[x] in drug_nodes_intersect_validate])
        IDX_validate_diff = np.array([x for x in range(len(drug_nodes_intersect_all)) if drug_nodes_intersect_all[x] in drug_nodes_intersect_validate_diff])
    
    m,n = matrix.shape # number of side effect # number of drug


    random.seed(1949) # for dataset split
    np.random.seed(1949) # for matrix initialization
    start_time = time.time()



    FOLDS = 5
    innerFOLDS = 4
    ####for test sets####
    setvar_cv(FOLDS)

    sz = n
    IDX = list(range(sz))
    fsz = int(sz/FOLDS)
    random.shuffle(IDX)
    IDX = np.array(IDX)
    offset = 0

    innersz = sz - fsz
    innerIDX = list(range(innersz))
    random.shuffle(innerIDX)
    innerIDX = np.array(innerIDX)
    innerfsz = int(innersz / innerFOLDS)
    inneroffset = 0
    # setvar_cv(FOLDS=FOLDS)

    if Validation == "nested_cv":
        print("---------- nested cv start ----------")
        for f in range(FOLDS):  # range(FOLDS):
            offset = 0 + fsz*f 
            idx_test = IDX[offset:offset + fsz]
    
            idx_train = IDX[np.setdiff1d(np.arange(len(IDX)), np.arange(offset,offset + fsz))]
            print("Fold:",f)
                
            innermatrix = matrix[:, idx_train]
    
            innerfeatureMat1 = featureMat1[idx_train, :].copy()
            innerfeatureMat2 = featureMat2[idx_train, :].copy()
    
            setvar_besttune(innerFOLDS)
    
            for innerf in range(innerFOLDS):
                inneroffset = 0 + innerf*innerfsz
                idx_test_inner = innerIDX[inneroffset:inneroffset + innerfsz]
                idx_train_inner = innerIDX[np.array(np.setdiff1d(np.arange(len(idx_train)), np.arange(inneroffset,inneroffset + innerfsz)))]
    
                print("Inner Fold:", innerf)


                
                if method_option == "KRNMF1":
                    lmd1 = (10**np.arange(-3, 1, 1, dtype=float)).tolist()
                    lmd2 = (np.arange(0, 1, 1, dtype=float)).tolist()
                    sigma = (10**np.arange(0, 3, 1, dtype=float)).tolist()
                    comp = np.arange(5, 25, 5, dtype=int).tolist()
                elif method_option == "KRNMF2":
                    lmd1 = (10**np.arange(-4, 2, 1, dtype=float)).tolist()
                    lmd2 = (np.arange(0, 1, 1, dtype=float)).tolist()
                    sigma = (10**np.arange(0, 5, 1, dtype=float)).tolist()
                    comp = np.arange(5, 30, 5, dtype=int).tolist()
                if method_option == "KRNMF1&2":
                    lmd1 = (10**np.arange(-3, 1, 1, dtype=float)).tolist()
                    lmd2 = (np.arange(0, 1, 1, dtype=float)).tolist()
                    sigma = (10**np.arange(0, 3, 1, dtype=float)).tolist()
                    comp = np.arange(10, 45, 5, dtype=int).tolist()
                    
                hyperparList = list(product(lmd1, lmd2, sigma, comp))

    
                setvar_tune(len(hyperparList))
    
                with parallel_backend('threading'):
                    Parallel(n_jobs=5)(delayed(tuning_loop)(innermatrix = innermatrix, idx_train_inner = idx_train_inner, \
                            idx_test_inner = idx_test_inner, feature_matrix_inner1 = innerfeatureMat1, feature_matrix_inner2 = innerfeatureMat2, \
                                hyperparList = hyperparList, i = i) \
                                    for i in range(len(hyperparList)))
    
                # tuning_plot(tuneVar=C, tune="C")
                hyperpars, evalValue = tuning_results(tuneVar=hyperparList)
    
    
                asg_besttune(innerf, value=evalValue, var=hyperpars)
                    
            _, bestHyperPars = besttune()
    
            print("--- tuning end ---")
            l1, l2, s, k = bestHyperPars
            idx_target = idx_test
            print('target size:', len(idx_target))
    
            print("------ lmdKR: ", l1, "lmdGNMF: ", l2, "sigma: ", s, "k: ", k, "------")
    
            results = fold(idx_target,idx_test,featureMat1,featureMat2,matrix,l1=l1,l2=l2,s=s,k=k)
            asgvar_cv(f=f, results=results)

        out_mean, out = cv_results()
        return out_mean, out

    elif Validation == "cv":
        print("---------- cv start ----------")
        setvar_besttune(FOLDS)

        for f in range(FOLDS):
            offset = 0 + fsz*f 
            idx_test = IDX[offset:offset + fsz]
            idx_train = IDX[np.setdiff1d(np.arange(len(IDX)), np.arange(offset,offset + fsz))]

            print("Fold:", f)
            if method_option == "KRNMF1":
                lmd1 = (10**np.arange(-3, 1, 1, dtype=float)).tolist()
                lmd2 = (np.arange(0, 1, 1, dtype=float)).tolist()
                sigma = (10**np.arange(0, 3, 1, dtype=float)).tolist()
                comp = np.arange(5, 40, 5, dtype=int).tolist()
            elif method_option == "KRNMF2":
                lmd1 = (10**np.arange(-3, 2, 1, dtype=float)).tolist()
                lmd2 = (np.arange(0, 1, 1, dtype=float)).tolist()
                sigma = (10**np.arange(0, 4, 1, dtype=float)).tolist()
                comp = np.arange(5, 30, 5, dtype=int).tolist()
            if method_option == "KRNMF1&2":
                lmd1 = (10**np.arange(-3, 2, 1, dtype=float)).tolist()
                lmd2 = (np.arange(0, 1, 1, dtype=float)).tolist()
                sigma = (10**np.arange(0, 3, 1, dtype=float)).tolist()
                comp = np.arange(5, 40, 5, dtype=int).tolist()
            hyperparList = list(product(lmd1, lmd2, sigma, comp))

            setvar_tune(len(hyperparList))
    
            with parallel_backend('threading'):
                Parallel(n_jobs=5)(delayed(tuning_loop)(innermatrix = matrix, idx_train_inner = idx_train, \
                        idx_test_inner = idx_test, feature_matrix_inner1 = featureMat1, feature_matrix_inner2 = featureMat2, \
                            hyperparList = hyperparList, i = i) \
                                for i in range(len(hyperparList)))
    
            hyperpars, evalValue = tuning_results(tuneVar=hyperparList)
            asg_besttune(f, value=evalValue, var=hyperpars)

        print("--- tuning end ---")
        # cv_results()
        _, bestHyperPars = besttune()
    elif Validation == "Validation":

        # validation
        idx_test = IDX_validate
        idx_train = IDX_validate_diff
        idx_target = idx_test
        print('target size:', len(idx_target))
        print("------ lmdKR: ", l1, "lmdNMF: ", l2, "sigma: ", s, "k: ", k, "------")
        results = fold(idx_target,idx_test,featureMat1_all,featureMat2_all,matrix_all,l1=l1,l2=l2,s=s,k=k)
        return
    elif Validation == "plot":

        # validation
        idx_test = IDX_validate
        idx_train = IDX_validate_diff
        idx_target = idx_test
        print('target size:', len(idx_target))
        print("------ lmdKR: ", l1, "lmdNMF: ", l2, "sigma: ", s, "k: ", k, "------")
        pr, roc = plotfold(idx_target,idx_test,featureMat1_all,featureMat2_all,matrix_all,l1=l1,l2=l2,s=s,k=k)
        return pr, roc

## 2. Nested CV and CV for VKR

### 2.1. Nested CV

Running the nested CV for feature DGI

In [ ]:
results_KRNMF1_mean, results_KRNMF1 = main(method_option="KRNMF1",Validation="nested_cv")

Running the nested CV for feature Chem

In [ ]:
results_KRNMF2_mean, results_KRNMF2 = main(method_option="KRNMF2",Validation="nested_cv")

Running the nested CV for integrated features

In [ ]:
results_KRNMF1_2_mean, results_KRNMF1_2 = main(method_option="KRNMF1&2",Validation="nested_cv")

### 2.2. CV to tune hyperparameters for independent test set

Running CV for DGI. The best hyperparameters are $\lambda=0.01, \sigma=10, k=20$.

In [ ]:
main(method_option="KRNMF1",Validation="cv") # 0.01 10 20

Running CV for Chem. The best hyperparameters are $\lambda=0.1, \sigma=10, k=20$.

In [ ]:
main(method_option="KRNMF2",Validation="cv") # 0.1 10 20

Running CV for DGI and Chem. The best hyperparameters are $\lambda=0.01, \sigma=10, k=25$.

In [ ]:
main(method_option="KRNMF1&2",Validation="cv") # 0.01 10 25

### 2.3. Independent test set

DGI

In [ ]:
main(method_option="KRNMF1", Validation="Validation", l1=0.01, l2=0, s=10, k=20)

Chem

In [ ]:
main(method_option="KRNMF2", Validation="Validation", l1=0.1, l2=0, s=10, k=20) 

Integration of DGI and Chem

In [ ]:
main(method_option="KRNMF1&2", Validation="Validation", l1=0.01, l2=0, s=10, k=25) 

### * Save data for PR and ROC

In [ ]:
KRNMF1_pr, KRNMF1_roc = \
    main(method_option="KRNMF1", Validation="plot", l1=0.01, l2=0, s=10, k=20)          
KRNMF1_pr.T.to_csv("Figs/KRNMF1_pr.csv", index=False)
KRNMF1_roc.T.to_csv("Figs/KRNMF1_roc.csv", index=False)

In [ ]:
KRNMF2_pr, KRNMF2_roc = \
    main(method_option="KRNMF2", Validation="plot", l1=0.1, l2=0, s=10, k=20)
KRNMF2_pr.T.to_csv("Figs/KRNMF2_pr.csv", index=False)
KRNMF2_roc.T.to_csv("Figs/KRNMF2_roc.csv", index=False)

In [ ]:
KRNMF12_pr, KRNMF12_roc = \
    main(method_option="KRNMF1&2", Validation="plot", l1=0.01, l2=0, s=10, k=25)
KRNMF12_pr.T.to_csv("Figs/KRNMF12_pr.csv", index=False)
KRNMF12_roc.T.to_csv("Figs/KRNMF12_roc.csv", index=False)